In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
train=pd.read_csv('/kaggle/input/indian-dance-form-recognition/dataset/train.csv')
test=pd.read_csv('/kaggle/input/indian-dance-form-recognition/dataset/test.csv')
train.head()

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [4]:
test.head()

,Image
0,508.jpg
1,246.jpg
2,473.jpg
3,485.jpg
4,128.jpg


In [5]:
print(train['target'].unique())

['manipuri' 'bharatanatyam' 'odissi' 'kathakali' 'kathak' 'sattriya'
 'kuchipudi' 'mohiniyattam']


In [6]:
Class_map={'manipuri':0, 'bharatanatyam':1, 'odissi':2 ,'kathakali':3, 'kathak':4, 'sattriya':5,
 'kuchipudi':6, 'mohiniyattam':7}
inverse_map={0:'manipuri', 1:'bharatanatyam', 2:'odissi' ,3:'kathakali',4: 'kathak', 5:'sattriya',
 6:'kuchipudi', 7:'mohiniyattam'}
train['target']=train['target'].map(Class_map)

In [7]:
train.head()

,Image,target
0,96.jpg,0
1,163.jpg,1
2,450.jpg,2
3,219.jpg,3
4,455.jpg,2


In [8]:
img_h,img_w= (224,224)

In [9]:
train_img=[]
train_label=[]
j=0
path='/kaggle/input/indian-dance-form-recognition/dataset/train'
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(img_h,img_w))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['target'][j])
    j=j+1

100%|██████████| 364/364 [00:03<00:00, 113.99it/s]


In [10]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train_img, train_label, test_size=0.30, shuffle= True)

In [11]:
test_img=[]
path='/kaggle/input/indian-dance-form-recognition/dataset/test'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(img_h,img_w))
    img=img.astype('float32')
    test_img.append(img)

100%|██████████| 156/156 [00:01<00:00, 124.07it/s]


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,# divide each input by its std
        rescale=1./255,
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

test_datagen= ImageDataGenerator(rescale=1./255)
valid_datagen= ImageDataGenerator(rescale=1./255)
train_datagen.fit(x_train)
test_datagen.fit(test_img)
valid_datagen.fit(x_valid)

In [13]:
train_img=np.array(train_img)
x_train= np.array(x_train)
x_valid= np.array(x_valid)
y_train= np.array(y_train)
y_valid= np.array(y_valid)
test_img=np.array(test_img)
train_label=np.array(train_label)
print("Shape of training data=",x_train.shape," and shape of labels of training data= ",y_train.shape)
print("Shape of validation data=",x_valid.shape," and shape of labels of validation data= ",y_valid.shape)
print("Shape of test data=",test_img.shape)

Shape of training data= (254, 224, 224, 3)  and shape of labels of training data=  (254,)
Shape of validation data= (110, 224, 224, 3)  and shape of labels of validation data=  (110,)
Shape of test data= (156, 224, 224, 3)


In [14]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Dropout

base_model= InceptionResNetV2(include_top=False, weights='imagenet', 
                              input_tensor=None, input_shape=(img_h,img_w,3), pooling='avg')

219062272/219055592 [==============================] - 1s 0us/step


In [15]:
'''for layer in base_model.layers[:-10]:
    layer.trainable=False'''
base_model.trainable=False
    
base_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
________________________________________________________________________________

activation_196 (Activation)     (None, 5, 5, 192)    0           batch_normalization_196[0][0]    
__________________________________________________________________________________________________
conv2d_197 (Conv2D)             (None, 5, 5, 224)    129024      activation_196[0][0]             
__________________________________________________________________________________________________
batch_normalization_197 (BatchN (None, 5, 5, 224)    672         conv2d_197[0][0]                 
__________________________________________________________________________________________________
activation_197 (Activation)     (None, 5, 5, 224)    0           batch_normalization_197[0][0]    
__________________________________________________________________________________________________
conv2d_195 (Conv2D)             (None, 5, 5, 192)    399360      block8_8_ac[0][0]                
__________________________________________________________________________________________________
conv2d_198

In [16]:
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau


model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(8,activation='softmax'))


from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop


#NASnet.trainable=False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]
    


model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
flatten (Flatten)            (None, 1536)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
batch_normalization_203 (Bat (None, 1536)              6144      
_________________________________________________________________
dense (Dense)                (None, 512)               786944    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_204 (Bat (None, 512)               2

In [17]:
model.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=32),
                    epochs=10,
          callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=32),
          verbose=1
             )

Epoch 1/10
8/8 [==============================] - 8s 939ms/step - loss: 2.5527 - accuracy: 0.2126 - val_loss: 1.7937 - val_accuracy: 0.3273 - lr: 0.0010
Epoch 2/10
8/8 [==============================] - 3s 422ms/step - loss: 1.9291 - accuracy: 0.4055 - val_loss: 1.7295 - val_accuracy: 0.3909 - lr: 0.0010
Epoch 3/10
8/8 [==============================] - 3s 385ms/step - loss: 1.4796 - accuracy: 0.4921 - val_loss: 1.4728 - val_accuracy: 0.4727 - lr: 0.0010
Epoch 4/10
8/8 [==============================] - 3s 392ms/step - loss: 1.3554 - accuracy: 0.6102 - val_loss: 1.3698 - val_accuracy: 0.5182 - lr: 0.0010
Epoch 5/10
8/8 [==============================] - 3s 427ms/step - loss: 1.3042 - accuracy: 0.5866 - val_loss: 1.3681 - val_accuracy: 0.5091 - lr: 0.0010
Epoch 6/10
8/8 [==============================] - 3s 391ms/step - loss: 1.2794 - accuracy: 0.5709 - val_loss: 1.3601 - val_accuracy: 0.5727 - lr: 0.0010
Epoch 7/10
8/8 [==============================] - 4s 439ms/step - loss: 1.2425 - a

Model_2 is resnetV2

In [18]:
from tensorflow.keras.applications.resnet import ResNet50
base_model_2= ResNet50(include_top=False, weights='imagenet',input_shape=(img_h,img_w,3), pooling='max')

'''for layer in base_model_2.layers[:-3]:
    layer.trainable=False'''
base_model_2.trainable=False
    
model_2=Sequential()
model_2.add(base_model_2)
model_2.add(Flatten())
model_2.add(Dropout(0.4))
model_2.add(BatchNormalization())

model_2.add(Dense(768, activation='relu'))
model_2.add(Dropout(0.2))
model_2.add(BatchNormalization())

model_2.add(Dense(256, activation='relu'))
model_2.add(Dropout(0.1))
model_2.add(BatchNormalization())


model_2.add(Dense(8,activation='softmax'))

model_2.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_2.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
batch_normalization_205 (Bat (None, 2048)              8192      
_________________________________________________________________
dense_2 (Dense)              (None, 768)               1573632   
_________________________________________________________________
dropout_3 (Dropout)          (None, 768)               0         
_______________________________________________________

In [19]:
model_2.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=32),
                    epochs=10,
          callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=32),
          verbose=1
             )

Epoch 1/10
8/8 [==============================] - 4s 522ms/step - loss: 2.9995 - accuracy: 0.1102 - val_loss: 2.3447 - val_accuracy: 0.1273 - lr: 0.0010
Epoch 2/10
8/8 [==============================] - 3s 369ms/step - loss: 2.6797 - accuracy: 0.1654 - val_loss: 2.1718 - val_accuracy: 0.1273 - lr: 0.0010
Epoch 3/10
8/8 [==============================] - 3s 379ms/step - loss: 2.5827 - accuracy: 0.1850 - val_loss: 2.1220 - val_accuracy: 0.2182 - lr: 0.0010
Epoch 4/10
8/8 [==============================] - 3s 386ms/step - loss: 2.5466 - accuracy: 0.1890 - val_loss: 2.0578 - val_accuracy: 0.2182 - lr: 0.0010
Epoch 5/10
8/8 [==============================] - 3s 336ms/step - loss: 2.2774 - accuracy: 0.2520 - val_loss: 2.0205 - val_accuracy: 0.2364 - lr: 0.0010
Epoch 6/10
8/8 [==============================] - 3s 345ms/step - loss: 2.3415 - accuracy: 0.2362 - val_loss: 2.0010 - val_accuracy: 0.3000 - lr: 0.0010
Epoch 7/10
8/8 [==============================] - 3s 341ms/step - loss: 2.2164 - a

Model_3 is VGG19

In [20]:
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
base_model_3=VGG19(include_top=False, weights='imagenet',input_shape=(img_h,img_w,3), pooling='max')

for layer in base_model_3.layers[:-4]:
    layer.trainable=False
#base_model_3.trainable=False
    
model_3=Sequential()
model_3.add(base_model_3)
model_3.add(Flatten())

model_3.add(BatchNormalization())
model_3.add(Dropout(0.2))


model_3.add(Dense(256, activation='relu'))
model_3.add(BatchNormalization())


model_3.add(Dense(64, activation='relu'))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.2))



model_3.add(Dense(8,activation='softmax'))

model_3.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_3.summary()

80142336/80134624 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 512)               20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
batch_normalization_208 (Bat (None, 512)               2048      
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_209 (Bat (None, 256)               1024      
_______________________________________________________

In [21]:
model_3.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=32),
                    epochs=20,
          callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=32),
          verbose=1
             )

Epoch 1/20
8/8 [==============================] - 4s 510ms/step - loss: 2.3661 - accuracy: 0.2559 - val_loss: 3.3501 - val_accuracy: 0.2636 - lr: 0.0010
Epoch 2/20
8/8 [==============================] - 3s 370ms/step - loss: 1.3522 - accuracy: 0.5354 - val_loss: 4.9994 - val_accuracy: 0.2909 - lr: 0.0010
Epoch 3/20
8/8 [==============================] - 3s 390ms/step - loss: 1.2242 - accuracy: 0.5669 - val_loss: 5.7297 - val_accuracy: 0.4273 - lr: 0.0010
Epoch 4/20
8/8 [==============================] - 3s 358ms/step - loss: 1.0411 - accuracy: 0.7047 - val_loss: 7.2945 - val_accuracy: 0.2455 - lr: 0.0010
Epoch 5/20
8/8 [==============================] - 3s 400ms/step - loss: 1.0240 - accuracy: 0.6417 - val_loss: 5.9017 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 6/20
8/8 [==============================] - 3s 406ms/step - loss: 0.7924 - accuracy: 0.7165 - val_loss: 4.1930 - val_accuracy: 0.2545 - lr: 0.0010
Epoch 7/20
8/8 [==============================] - 3s 403ms/step - loss: 0.6588 - a

Stacking ensemble

In [22]:
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate
def stacking_ensemble(members,input_shape,n_classes):
  commonInput = Input(shape=input_shape)
  out=[]

  for model in members:
   
    model._name= model.get_layer(index = 0)._name +"-test"+ str(members.index(model)+1)
    out.append(model(commonInput))

  modeltmp = concatenate(out,axis=-1)
  modeltmp = Dense(32, activation='relu')(modeltmp)
  modeltmp = Dense(16, activation='relu')(modeltmp)
  modeltmp = Dense(n_classes, activation='softmax')(modeltmp)
  stacked_model = Model(commonInput,modeltmp)
  stacked_model.compile( loss='categorical_crossentropy',optimizer= 'adam', metrics=['accuracy'])

  return stacked_model

In [23]:
members=[model,model_2,model_3]

In [24]:
stacked_model= stacking_ensemble(members,(img_h,img_w,3),8)
stacked_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
inception_resnet_v2-test1 (Sequ (None, 8)            55135976    input_4[0][0]                    
__________________________________________________________________________________________________
resnet50-test2 (Sequential)     (None, 8)            25372552    input_4[0][0]                    
__________________________________________________________________________________________________
vgg19-test3 (Sequential)        (None, 8)            20176008    input_4[0][0]                    
______________________________________________________________________________________________

In [25]:
stacked_model.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=32),
                    epochs=10,
          callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=32),
          verbose=1
             )

Epoch 1/10
8/8 [==============================] - 10s 1s/step - loss: 2.0420 - accuracy: 0.2717 - val_loss: 2.0121 - val_accuracy: 0.3091 - lr: 0.0010
Epoch 2/10
8/8 [==============================] - 4s 502ms/step - loss: 2.0044 - accuracy: 0.3504 - val_loss: 1.9891 - val_accuracy: 0.4091 - lr: 0.0010
Epoch 3/10
8/8 [==============================] - 4s 507ms/step - loss: 1.9659 - accuracy: 0.3976 - val_loss: 1.9724 - val_accuracy: 0.4273 - lr: 0.0010
Epoch 4/10
8/8 [==============================] - 4s 503ms/step - loss: 1.9394 - accuracy: 0.4370 - val_loss: 1.9797 - val_accuracy: 0.3273 - lr: 0.0010
Epoch 5/10
8/8 [==============================] - 4s 508ms/step - loss: 1.8780 - accuracy: 0.4961 - val_loss: 1.9454 - val_accuracy: 0.3455 - lr: 0.0010
Epoch 6/10
8/8 [==============================] - 4s 542ms/step - loss: 1.8325 - accuracy: 0.4961 - val_loss: 1.8471 - val_accuracy: 0.4364 - lr: 0.0010
Epoch 7/10
8/8 [==============================] - 4s 497ms/step - loss: 1.7815 - acc

In [26]:
labels = stacked_model.predict(test_img)
print(labels[:4])
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
print(class_label[:3])

[[0.09978335 0.1535989  0.09640347 0.07053228 0.08701309 0.13617492
  0.07505257 0.28144145]
 [0.09978335 0.1535989  0.09640347 0.07053228 0.08701309 0.13617492
  0.07505257 0.28144145]
 [0.09978335 0.1535989  0.09640347 0.07053228 0.08701309 0.13617492
  0.07505257 0.28144145]
 [0.09978335 0.1535989  0.09640347 0.07053228 0.08701309 0.13617492
  0.07505257 0.28144145]]
['mohiniyattam', 'mohiniyattam', 'mohiniyattam']


# So our stacked model is performing very well..